In [24]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Bidirectional, Dropout, Input

# Parameters
sequence_length = 50  # Length of each sequence
num_samples = 10000   # Number of samples
batch_size = 32       # Batch size for training
epochs = 5           # Number of epochs

def levels(x):
    if x > 0.3: return 2
    if x < -0.3: return 0
    return 1

# Generate sine wave data
def generate_sine_wave_data(num_samples, sequence_length):
    x = np.linspace(0, 10 * np.pi, num_samples * sequence_length)  # Generate x values
    sine_wave = np.sin(x)  # Compute sin(x)
    sine_wave = sine_wave.reshape(num_samples, sequence_length, 1)  # Reshape to (num_samples, sequence_length, 1)
    return sine_wave

# Generate encoder input, decoder input, and decoder target
def generate_seq2seq_data(num_samples, sequence_length):
    # Generate sine wave data
    sine_wave = generate_sine_wave_data(num_samples, sequence_length)
    
    # Encoder input: sin(x) values
    encoder_input = sine_wave
    
    # Decoder target: same as encoder input
    decoder_target = pd.DataFrame(sine_wave).applymap(levels)
    
    # Decoder input: shifted version of decoder target (for teacher forcing)
    decoder_input = np.zeros_like(decoder_target)
    decoder_input[:, 1:, :] = decoder_target[:, :-1, :]  # Shift right by one timestep
    
    return pd.DataFrame(encoder_input), pd.DataFrame(decoder_input), decoder_target

# Generate data
encoder_input, decoder_input, decoder_target = generate_seq2seq_data(num_samples, sequence_length)

print(decoder_target[:10])
print(decoder_target.shape)

# Define the seq2seq model
def create_model1():
    # Define the encoder
    encoder_inputs = Input(shape=(None, 1))
    encoder = Bidirectional(LSTM(64, return_sequences=True))(encoder_inputs)
    encoder = Dropout(0.2)(encoder)
    encoder_outputs, state_h, state_c = LSTM(64, return_state=True)(encoder)
    encoder_states = [state_h, state_c]

    # Define the decoder
    decoder_inputs = Input(shape=(None, 3))  # 3 classes: 0, 1, 2
    decoder_lstm = LSTM(64, return_sequences=True, return_state=True)

    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_outputs = Dropout(0.2)(decoder_outputs)
    decoder_dense = TimeDistributed(Dense(3, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'recall'])

    return model

model = create_model1()

# Compile the model
# model.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'recall'])  # Mean Squared Error for regression

# Print model summary
model.summary()



ValueError: Must pass 2-d input. shape=(10000, 50, 1)

In [20]:
test_decoder_input

array([[[ 0.        ],
        [ 0.        ],
        [ 0.59811053],
        [ 0.95866785],
        [ 0.93846842],
        [ 0.5455349 ],
        [-0.06407022],
        [-0.6482284 ],
        [-0.97492791],
        [-0.91441262],
        [-0.49071755],
        [ 0.12787716],
        [ 0.69568255],
        [ 0.98718178],
        [ 0.88659931],
        [ 0.43388374],
        [-0.19115863],
        [-0.740278  ],
        [-0.99537911],
        [-0.85514276],
        [-0.375267  ],
        [ 0.25365458],
        [ 0.78183148],
        [ 0.99948622],
        [ 0.82017225],
        [ 0.31510822],
        [-0.31510822],
        [-0.82017225],
        [-0.99948622],
        [-0.78183148],
        [-0.25365458],
        [ 0.375267  ],
        [ 0.85514276],
        [ 0.99537911],
        [ 0.740278  ],
        [ 0.19115863],
        [-0.43388374],
        [-0.88659931],
        [-0.98718178],
        [-0.69568255],
        [-0.12787716],
        [ 0.49071755],
        [ 0.91441262],
        [ 0

In [ ]:
# Train the model
history = model.fit(
    [encoder_input, decoder_input],  # Inputs: encoder_input and decoder_input
    decoder_target,                  # Target: decoder_target
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

# Test the model on a new sine wave
test_encoder_input, test_decoder_input, test_decoder_target = generate_seq2seq_data(1, sequence_length)
predicted_output = model.predict([test_encoder_input, test_decoder_input])

# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(test_decoder_target[0, :, 0]+1, label='True Output')
plt.plot(predicted_output[0, :, 0], label='Predicted Output')
plt.legend()
plt.title('True vs Predicted Output')
plt.show()